In [1]:
import numpy as np
from patsy import dmatrices
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.formula.api as sm# synthetic sample data
import seaborn as sns

%matplotlib widget

In [4]:
path_save = r"C:\Users\coren\Dropbox\Apps\Snellius2\data_tables\time_plate_infos.pick"

time_plate_info = pd.read_pickle(path_save)


In [ ]:
table.groupby()

In [5]:
plates = [76,94,102,152,792,799,800,808,91,80]

# table = time_plate_info.loc[time_plate_info['Plate']==792]
table = time_plate_info.loc[time_plate_info['Plate'].isin(plates)]

rhos = []
ns = []
ans_rate = []
stop_rate = []
branch_rate = []
ts = []
speeds = []
bas_rho = []
ROI = []
for index in range(1,20):
    column = f"ring_density_incr-100_index-{index}"
    rhos+= list(table[column])
    column = f"ring_active_tips_density_incr-100_index-{index}"
    ns+= list(table[column])
    column = f"ring_anas_density_incr-100_index-{index}"
    ans_rate+= list(table[column])
    column = f"ring_branch_density_incr-100_index-{index}"
    branch_rate+= list(table[column])
    column = f"ring_stop_density_incr-100_index-{index}"
    stop_rate+= list(table[column])
    column = f"t"
    ts+= list(table[column])
    column = f"mean_speed_incr-100_index-{index}"
    speeds+= list(table[column])
    column = f"ring_bas_density_incr-100_index-{index}"
    bas_rho+= list(table[column])
    column = f"out_study"
    ROI+= list(table[column])

In [15]:
data = np.array((rhos,ns,ans_rate,stop_rate,branch_rate,ts,speeds,bas_rho,ROI)).transpose()
table_rhos = pd.DataFrame(data,columns=("rho","n","an_rate","stop",'branch','ts','mean_speed','bas_rho','in_ROI'))
table_rhos['nrho'] = table_rhos['n']*table_rhos['rho']
table_rhos = table_rhos.dropna()
table_rhos['n2'] = table_rhos['n']**2
table_rhos['rho2'] = table_rhos['rho']**2
table_rhos = table_rhos.astype(float)
table_rhos = table_rhos.loc[(table_rhos['n']!=0)]
table_rhos = table_rhos.loc[(table_rhos['mean_speed']<=400)]

table_rhos['bin_n'] = (table_rhos['n']*100).astype(int)/100
table_rhos['bin_nrho'] = (table_rhos['nrho']/5).astype(int)*5

In [23]:
columns = ["rho","n","an_rate","stop",'branch','mean_speed','bas_rho','nrho']
corr = table_rhos[columns].corr()
corr.style.background_gradient(cmap='coolwarm')

KeyError: "['mean_speed', 'bas_rho'] not in index"

In [6]:
fig, ax = plt.subplots()
dataplot = sns.heatmap(corr, cmap="YlGnBu", annot=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
fig, ax = plt.subplots()
table_rhos.plot('n','mean_speed',kind ="scatter",ax = ax,alpha = 0.3)
fit = 0.03
# ax.plot(table_rhos['n'],fit*table_rhos['n'],label = f'beta = {fit*24} day-1',color="red")
ax.set_ylabel('branching rate ($h^{-1}$)')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


In [16]:
alphas = []
boot_interc = []
n_boots = 300
n_points = len(table_rhos)
x = 'n'
y = "branch"
ols_model = sm.ols(formula = f"{y} ~ {x}", data=table_rhos)
results = ols_model.fit()# coefficients
print('Intercept, x-Slope : {}'.format(results.params))
y_pred = ols_model.fit().predict(table_rhos[x].astype(float))# plot results
fig, ax = plt.subplots()
for _ in range(n_boots):
    # sample the rows, same size, with replacement
    sample_df = table_rhos.sample(n=n_points, replace=True)
    # fit a linear regression
    ols_model_temp = sm.ols(formula = f"{y} ~ {x}", data=sample_df)
    results_temp = ols_model_temp.fit()

    # append coefficients
    boot_interc.append(results_temp.params[0])
    alphas.append(results_temp.params[1])

    # plot a greyed out line
    y_pred_temp = ols_model_temp.fit().predict(sample_df[x].astype(float))
    ax.plot(sample_df[x], y_pred_temp, color="grey", alpha=0.1)# add data points
ax.scatter(table_rhos[x], table_rhos[y])
# ax.plot(table_rhos[x], y_pred, linewidth=2)
data = table_rhos.groupby(f'bin_{x}')[y]
ax.errorbar(data.mean().index,data.mean(),color = "red",yerr=data.std()/np.sqrt(data.count()))
ax.grid(True)
ax.set_ylabel("density of new tips ($h^{-1}.mm^{-2}$)")
ax.set_xlabel("density of tips ($mm^{-2}$)")
# ax.set_xlim((0,0.125))


Intercept, x-Slope : Intercept   -0.000637
n            0.044220
dtype: float64


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'density of tips ($mm^{-2}$)')

In [17]:
fig, ax = plt.subplots()

plates = [76,94,102,152,792,799,800,808,91,80]
# plates = [76,94,102,152,792]

for plate in plates:
    table = time_plate_info.loc[time_plate_info['Plate']==plate]
    rhos = []
    ns = []
    ans_rate = []
    stop_rate = []
    branch_rate = []
    ts = []
    for index in range(1,20):
        column = f"ring_density_incr-100_index-{index}"
        rhos+= list(table[column])
        column = f"ring_active_tips_density_incr-100_index-{index}"
        ns+= list(table[column])
        column = f"ring_anas_density_incr-100_index-{index}"
        ans_rate+= list(table[column])
        column = f"ring_branch_density_incr-100_index-{index}"
        branch_rate+= list(table[column])
        column = f"ring_stop_density_incr-100_index-{index}"
        stop_rate+= list(table[column])
        column = f"t"
        ts+= list(table[column])
    data = np.array((rhos,ns,ans_rate,stop_rate,branch_rate,ts)).transpose()
    table_rhos = pd.DataFrame(data,columns=("rho","n","an_rate","stop",'branch','ts'))
    table_rhos['nrho'] = table_rhos['n']*table_rhos['rho']
    table_rhos = table_rhos.dropna()
    table_rhos['n2'] = table_rhos['n']**2
    table_rhos['rho2'] = table_rhos['rho']**2
    table_rhos = table_rhos.astype(float)
    table_rhos = table_rhos.loc[(table_rhos['n']!=0)]
    table_rhos['bin_n'] = (table_rhos['n']*100).astype(int)/100
    table_rhos['bin_nrho'] = (table_rhos['nrho']/5).astype(int)*5
    data = table_rhos.groupby('bin_n')['branch'].mean()
    ax.errorbar(data.index,data,yerr=table_rhos.groupby('bin_n')['branch'].std()/np.sqrt(table_rhos.groupby('bin_n')['branch'].count()),label = plate)
ax.set_ylabel("density of new tips ($h^{-1}.mm^{-2}$)")
ax.set_xlabel("density of tips ($mm^{-2}$)")
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
fig, ax = plt.subplots()

table = time_plate_info.loc[time_plate_info['Plate'].isin(plates)]
rhos = []
ns = []
ans_rate = []
stop_rate = []
branch_rate = []
ts = []
for index in range(1,20):
    column = f"ring_density_incr-100_index-{index}"
    rhos+= list(table[column])
    column = f"ring_active_tips_density_incr-100_index-{index}"
    ns+= list(table[column])
    column = f"ring_anas_density_incr-100_index-{index}"
    ans_rate+= list(table[column])
    column = f"ring_branch_density_incr-100_index-{index}"
    branch_rate+= list(table[column])
    column = f"ring_stop_density_incr-100_index-{index}"
    stop_rate+= list(table[column])
    column = f"t"
    ts+= list(table[column])
data = np.array((rhos,ns,ans_rate,stop_rate,branch_rate,ts)).transpose()
table_rhos = pd.DataFrame(data,columns=("rho","n","an_rate","stop",'branch','ts'))
table_rhos['nrho'] = table_rhos['n']*table_rhos['rho']
table_rhos = table_rhos.dropna()
table_rhos['n2'] = table_rhos['n']**2
table_rhos['rho2'] = table_rhos['rho']**2
table_rhos = table_rhos.astype(float)
table_rhos = table_rhos.loc[(table_rhos['n']!=0)]
table_rhos['bin_n'] = (table_rhos['n']*100).astype(int)/100
table_rhos['bin_nrho'] = (table_rhos['nrho']/5).astype(int)*5
data = table_rhos.groupby('bin_n')['branch'].mean()
ax.errorbar(data.index,data,yerr=table_rhos.groupby('bin_n')['branch'].std()/np.sqrt(table_rhos.groupby('bin_n')['branch'].count()))
ax.set_ylabel("density of new tips ($h^{-1}.mm^{-2}$)")
ax.set_xlabel("density of tips ($mm^{-2}$)")
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


In [19]:
import seaborn as sns
fig, ax = plt.subplots()

sns.distplot(np.array(alphas)*24,ax = ax,axlabel = r"estimated $\alpha$ ($day^{-1}$)")
ax.vlines(1.3,0,8,color='red')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [13]:
5.5*24

132.0

In [15]:
np.std(alphas)*24

0.05380457878922196

In [20]:
betas = []
boot_interc = []
n_boots = 300
n_points = len(table_rhos)
x = 'nrho'
y = "an_rate"
ols_model = sm.ols(formula = f"{y} ~ {x}", data=table_rhos)
results = ols_model.fit()# coefficients
print('Intercept, x-Slope : {}'.format(results.params))
y_pred = ols_model.fit().predict(table_rhos[x].astype(float))# plot results
fig, ax = plt.subplots()
for _ in range(n_boots):
    # sample the rows, same size, with replacement
    sample_df = table_rhos.sample(n=n_points, replace=True)
    # fit a linear regression
    ols_model_temp = sm.ols(formula = f"{y} ~ {x}", data=sample_df)
    results_temp = ols_model_temp.fit()

    # append coefficients
    boot_interc.append(results_temp.params[0])
    betas.append(results_temp.params[1])

    # plot a greyed out line
    y_pred_temp = ols_model_temp.fit().predict(sample_df[x].astype(float))
    ax.plot(sample_df[x], y_pred_temp, color="grey", alpha=0.1)# add data points
ax.scatter(table_rhos[x], table_rhos[y])
data = table_rhos.groupby('bin_nrho')['an_rate']

ax.errorbar(data.mean().index,data.mean(),color = "red",yerr=data.std()/np.sqrt(data.count()))

# ax.plot(table_rhos[x], y_pred, linewidth=2)
ax.grid(True)
ax.set_xlabel(r"$n \rho$ ($\mu m.mm^{-4}$)")

ax.set_ylabel("density of anastomosis events ($h^{-1}.mm^{-2}$)")
ax.set_ylim((0,0.008))
ax.set_xlim((0,200))

Intercept, x-Slope : Intercept   -0.000039
nrho         0.000016
dtype: float64


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 200.0)

In [21]:
fig, ax = plt.subplots()

sns.distplot(np.array(betas),ax = ax,axlabel = r"estimated beta ($mm^{2}.\mu m^{-1}.day^{-1}$)")
ax.vlines(2.1e-5,0,250000,color='red')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [21]:
np.mean(betas)

1.5946048523106214e-05

In [62]:
plt.close("all")
betas2 = []
boot_interc = []
n_boots = 300
n_points = len(table_rhos)
x = 'n'
y = "an_rate"
ols_model = sm.ols(formula = f"{y} ~ {x}", data=table_rhos)
results = ols_model.fit()# coefficients
print('Intercept, x-Slope : {}'.format(results.params))
y_pred = ols_model.fit().predict(table_rhos[x].astype(float))# plot results
fig, ax = plt.subplots()
for _ in range(n_boots):
    # sample the rows, same size, with replacement
    sample_df = table_rhos.sample(n=n_points, replace=True)
    # fit a linear regression
    ols_model_temp = sm.ols(formula = f"{y} ~ {x}", data=sample_df)
    results_temp = ols_model_temp.fit()

    # append coefficients
    boot_interc.append(results_temp.params[0])
    betas2.append(results_temp.params[1])

    # plot a greyed out line
    y_pred_temp = ols_model_temp.fit().predict(sample_df[x].astype(float))
    ax.plot(sample_df[x], y_pred_temp, color="grey", alpha=0.1)# add data points
ax.scatter(table_rhos[x], table_rhos[y])
# ax.plot(table_rhos[x], y_pred, linewidth=2)
data = table_rhos.groupby(f'bin_{x}')[y]
ax.errorbar(data.mean().index,data.mean(),color = "red",yerr=data.std()/np.sqrt(data.count()))
ax.grid(True)
ax.set_xlabel(r"$n $ ($\mu m.mm^{-4}$)")
ax.set_ylabel("density of stopping events ($h^{-1}.mm^{-2}$)")
ax.set_ylim((0,0.006))


Intercept, x-Slope : Intercept     13.67232
n            514.18659
dtype: float64


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'density of stopping events ($h^{-1}.mm^{-2}$)')

In [22]:
2*np.mean(alphas)/(np.mean(betas))

5527.945856303682

In [63]:
plt.close("all")
betas2 = []
boot_interc = []
n_boots = 1
n_points = len(table_rhos)
x = 'n'
y = "an_rate"
ols_model = sm.ols(formula = f"{y} ~ {x}", data=table_rhos)
results = ols_model.fit()# coefficients
print('Intercept, x-Slope : {}'.format(results.params))
y_pred = ols_model.fit().predict(table_rhos[x].astype(float))# plot results
fig, ax = plt.subplots()
for _ in range(n_boots):
    # sample the rows, same size, with replacement
    sample_df = table_rhos.sample(n=n_points, replace=True)
    # fit a linear regression
    ols_model_temp = sm.ols(formula = f"{y} ~ {x}", data=sample_df)
    results_temp = ols_model_temp.fit()

    # append coefficients
    boot_interc.append(results_temp.params[0])
    betas2.append(results_temp.params[1])

    # plot a greyed out line
    y_pred_temp = ols_model_temp.fit().predict(sample_df[x].astype(float))
    ax.plot(sample_df[x], y_pred_temp, color="grey", alpha=0.1)# add data points
# ax.scatter(table_rhos[x], table_rhos[y])
# ax.plot(table_rhos[x], y_pred, linewidth=2)
data = table_rhos.groupby(f'bin_{x}')[y]
ax.errorbar(data.mean().index,data.mean(),color = "red",yerr=data.std()/np.sqrt(data.count()))
ax.grid(True)
ax.set_xlabel(r"$n $ ($\mu m.mm^{-4}$)")
ax2=ax.twiny()
ax.set_ylabel("density of stopping events ($h^{-1}.mm^{-2}$)")
ax.set_ylim((0,0.006))
betas2 = []
boot_interc = []
n_points = len(table_rhos)
x = 'nrho'
y = "an_rate"
ols_model = sm.ols(formula = f"{y} ~ {x}", data=table_rhos)
results = ols_model.fit()# coefficients
print('Intercept, x-Slope : {}'.format(results.params))
y_pred = ols_model.fit().predict(table_rhos[x].astype(float))# plot results
for _ in range(n_boots):
    # sample the rows, same size, with replacement
    sample_df = table_rhos.sample(n=n_points, replace=True)
    # fit a linear regression
    ols_model_temp = sm.ols(formula = f"{y} ~ {x}", data=sample_df)
    results_temp = ols_model_temp.fit()

    # append coefficients
    boot_interc.append(results_temp.params[0])
    betas2.append(results_temp.params[1])

    # plot a greyed out line
    y_pred_temp = ols_model_temp.fit().predict(sample_df[x].astype(float))
    ax2.plot(sample_df[x], y_pred_temp, color="grey", alpha=0.1)# add data points
# ax2.scatter(table_rhos[x], table_rhos[y])
# ax.plot(table_rhos[x], y_pred, linewidth=2)
data = table_rhos.groupby(f'bin_{x}')[y]
ax2.errorbar(data.mean().index,data.mean(),color = "green",yerr=data.std()/np.sqrt(data.count()))
ax.set_xlabel(r"$n $ ($mm^{-2}$)",color='red')
ax2.set_xlabel(r"$n \rho$ ($\mu m.mm^{-4}$)",color='green')


Intercept, x-Slope : Intercept    0.000204
n            0.007654
dtype: float64


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Intercept, x-Slope : Intercept   -0.000039
nrho         0.000016
dtype: float64


Text(0.5, 0, '$n \\rho$ ($\\mu m.mm^{-4}$)')

In [65]:
plt.close("all")
betas2 = []
boot_interc = []
n_boots = 1
n_points = len(table_rhos)
x = 'n'
y = "nrho"
ols_model = sm.ols(formula = f"{y} ~ {x}", data=table_rhos)
results = ols_model.fit()# coefficients
print('Intercept, x-Slope : {}'.format(results.params))
y_pred = ols_model.fit().predict(table_rhos[x].astype(float))# plot results
fig, ax = plt.subplots()
for _ in range(n_boots):
    # sample the rows, same size, with replacement
    sample_df = table_rhos.sample(n=n_points, replace=True)
    # fit a linear regression
    ols_model_temp = sm.ols(formula = f"{y} ~ {x}", data=sample_df)
    results_temp = ols_model_temp.fit()

    # append coefficients
    boot_interc.append(results_temp.params[0])
    betas2.append(results_temp.params[1])

    # plot a greyed out line
    y_pred_temp = ols_model_temp.fit().predict(sample_df[x].astype(float))
#     ax.plot(sample_df[x], y_pred_temp, color="grey", alpha=0.1)# add data points
ax.scatter(table_rhos[x], table_rhos[y])
# ax.plot(table_rhos[x], y_pred, linewidth=2)
data = table_rhos.groupby(f'bin_{x}')[y]
ax.errorbar(data.mean().index,data.mean(),color = "red",yerr=data.std()/np.sqrt(data.count()))
ax.grid(True)
ax.set_xlabel(r"$n $ ($mm^{-2}$)")
ax.set_ylabel(r"$n \rho $ ($\mu m.mm^{-2}$)")
# ax.set_ylim((0,0.006))


Intercept, x-Slope : Intercept     13.67232
n            514.18659
dtype: float64


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$n \\rho $ ($\\mu m.mm^{-2}$)')

In [60]:
fig, ax = plt.subplots()
ax.scatter(table_rhos['bin_n'],table_rhos['rho'],alpha=0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [130]:
np.mean(betas)*24

0.0006201528414848493

In [20]:
4*np.mean(alphas)/np.mean(betas)*np.sqrt((np.std(alphas)/np.mean(alphas))**2+(np.std(betas)/np.mean(betas))**2)

1150.1854768958208

In [28]:
fig, ax = plt.subplots()

plates = [76,94,102,152,792,799,800,808,91,80]
for plate in plates:
    table = time_plate_info.loc[time_plate_info['Plate']==plate]
    rhos = []
    ns = []
    ans_rate = []
    stop_rate = []
    branch_rate = []
    ts = []
    for index in range(1,20):
        column = f"ring_density_incr-100_index-{index}"
        rhos+= list(table[column])
        column = f"ring_active_tips_density_incr-100_index-{index}"
        ns+= list(table[column])
        column = f"ring_anas_density_incr-100_index-{index}"
        ans_rate+= list(table[column])
        column = f"ring_branch_density_incr-100_index-{index}"
        branch_rate+= list(table[column])
        column = f"ring_stop_density_incr-100_index-{index}"
        stop_rate+= list(table[column])
        column = f"t"
        ts+= list(table[column])
    data = np.array((rhos,ns,ans_rate,stop_rate,branch_rate,ts)).transpose()
    table_rhos = pd.DataFrame(data,columns=("rho","n","an_rate","stop",'branch','ts'))
    table_rhos['nrho'] = table_rhos['n']*table_rhos['rho']
    table_rhos = table_rhos.dropna()
    table_rhos['n2'] = table_rhos['n']**2
    table_rhos['rho2'] = table_rhos['rho']**2
    table_rhos = table_rhos.astype(float)
    table_rhos = table_rhos.loc[(table_rhos['n']!=0)]
    alphas = []
    boot_interc = []
    n_boots = 300
    n_points = 1000
    x = 'n'
    y = "branch"
    ols_model = sm.ols(formula = f"{y} ~ {x}", data=table_rhos)
    results = ols_model.fit()# coefficients
    print('Intercept, x-Slope : {}'.format(results.params))
    y_pred = ols_model.fit().predict(table_rhos[x].astype(float))# plot results
    for _ in range(n_boots):
        # sample the rows, same size, with replacement
        sample_df = table_rhos.sample(n=n_points, replace=True)
        # fit a linear regression
        ols_model_temp = sm.ols(formula = f"{y} ~ {x}", data=sample_df)
        results_temp = ols_model_temp.fit()

        # append coefficients
        boot_interc.append(results_temp.params[0])
        alphas.append(results_temp.params[1])

        # plot a greyed out line
        y_pred_temp = ols_model_temp.fit().predict(sample_df[x].astype(float))

    sns.distplot(np.array(alphas)*24,ax = ax,axlabel = r"estimated alpha ($day^{-1}$)",label=plate)
ax.set_xlim((0,2))
ax.set_ylim((0,6))

plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Intercept, x-Slope : Intercept   -0.001307
n            0.060162
dtype: float64


C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


Intercept, x-Slope : Intercept   -0.000798
n            0.046517
dtype: float64


C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


...

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


Intercept, x-Slope : Intercept   -0.000447
n            0.038817
dtype: float64


C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


Intercept, x-Slope : Intercept   -0.000523
n            0.035921
dtype: float64


C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [179]:
fig, ax = plt.subplots()

plates = [76,94,102,152,792,799,800,808,91,80]
for plate in plates:
    table = time_plate_info.loc[time_plate_info['Plate']==plate]
    rhos = []
    ns = []
    ans_rate = []
    stop_rate = []
    branch_rate = []
    ts = []
    for index in range(1,20):
        column = f"ring_density_incr-100_index-{index}"
        rhos+= list(table[column])
        column = f"ring_active_tips_density_incr-100_index-{index}"
        ns+= list(table[column])
        column = f"ring_anas_density_incr-100_index-{index}"
        ans_rate+= list(table[column])
        column = f"ring_branch_density_incr-100_index-{index}"
        branch_rate+= list(table[column])
        column = f"ring_stop_density_incr-100_index-{index}"
        stop_rate+= list(table[column])
        column = f"t"
        ts+= list(table[column])
    data = np.array((rhos,ns,ans_rate,stop_rate,branch_rate,ts)).transpose()
    table_rhos = pd.DataFrame(data,columns=("rho","n","an_rate","stop",'branch','ts'))
    table_rhos['nrho'] = table_rhos['n']*table_rhos['rho']
    table_rhos = table_rhos.dropna()
    table_rhos['n2'] = table_rhos['n']**2
    table_rhos['rho2'] = table_rhos['rho']**2
    table_rhos = table_rhos.astype(float)
    table_rhos = table_rhos.loc[(table_rhos['nrho']!=0)]

    betas = []
    boot_interc = []
    n_boots = 300
    x = 'nrho'
    y = "an_rate"
    ols_model = sm.ols(formula = f"{y} ~ {x}", data=table_rhos)
    results = ols_model.fit()# coefficients
    print('Intercept, x-Slope : {}'.format(results.params))
    y_pred = ols_model.fit().predict(table_rhos[x].astype(float))# plot results
    for _ in range(n_boots):
        # sample the rows, same size, with replacement
        sample_df = table_rhos.sample(n=n_points, replace=True)
        # fit a linear regression
        ols_model_temp = sm.ols(formula = f"{y} ~ {x}", data=sample_df)
        results_temp = ols_model_temp.fit()

        # append coefficients
        boot_interc.append(results_temp.params[0])
        betas.append(results_temp.params[1])

        # plot a greyed out line
        y_pred_temp = ols_model_temp.fit().predict(sample_df[x].astype(float))

    sns.distplot(np.array(betas)*24,ax = ax,axlabel = r"estimated beta ($mm^{2}.\mu m^{-1}.day^{-1}$)",label=plate)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Intercept, x-Slope : Intercept   -0.000021
nrho         0.000025
dtype: float64


C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


Intercept, x-Slope : Intercept   -0.000027
nrho         0.000021
dtype: float64


C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


...

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


Intercept, x-Slope : Intercept    0.000015
nrho         0.000016
dtype: float64


C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


Intercept, x-Slope : Intercept   -0.000006
nrho         0.000019
dtype: float64


C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [ ]:
fig, ax = plt.subplots()

plates = [76,94,102,152,792,799,800,808,91,80]
for plate in plates:
    table = time_plate_info.loc[time_plate_info['Plate']==plate]
    rhos = []
    ns = []
    ans_rate = []
    stop_rate = []
    branch_rate = []
    ts = []
    for index in range(1,20):
        column = f"ring_density_incr-100_index-{index}"
        rhos+= list(table[column])
        column = f"ring_active_tips_density_incr-100_index-{index}"
        ns+= list(table[column])
        column = f"ring_anas_density_incr-100_index-{index}"
        ans_rate+= list(table[column])
        column = f"ring_branch_density_incr-100_index-{index}"
        branch_rate+= list(table[column])
        column = f"ring_stop_density_incr-100_index-{index}"
        stop_rate+= list(table[column])
        column = f"t"
        ts+= list(table[column])
    data = np.array((rhos,ns,ans_rate,stop_rate,branch_rate,ts)).transpose()
    table_rhos = pd.DataFrame(data,columns=("rho","n","an_rate","stop",'branch','ts'))
    table_rhos['nrho'] = table_rhos['n']*table_rhos['rho']
    table_rhos = table_rhos.dropna()
    table_rhos['n2'] = table_rhos['n']**2
    table_rhos['rho2'] = table_rhos['rho']**2
    table_rhos = table_rhos.astype(float)
    table_rhos = table_rhos.loc[(table_rhos['n']!=0)]
    table_rhos['bin_n'] = (table_rhos['n']*100).astype(int)/100
    table_rhos['bin_nrho'] = (table_rhos['nrho']/5).astype(int)*5

    betas = []
    boot_interc = []
    n_boots = 300
    x = 'nrho'
    y = "an_rate"
    ols_model = sm.ols(formula = f"{y} ~ {x}", data=table_rhos)
    results = ols_model.fit()# coefficients
    print('Intercept, x-Slope : {}'.format(results.params))
    y_pred = ols_model.fit().predict(table_rhos[x].astype(float))# plot results
    for _ in range(n_boots):
        # sample the rows, same size, with replacement
        sample_df = table_rhos.sample(n=n_points, replace=True)
        # fit a linear regression
        ols_model_temp = sm.ols(formula = f"{y} ~ {x}", data=sample_df)
        results_temp = ols_model_temp.fit()

        # append coefficients
        boot_interc.append(results_temp.params[0])
        betas.append(results_temp.params[1])

        # plot a greyed out line
        y_pred_temp = ols_model_temp.fit().predict(sample_df[x].astype(float))

    sns.distplot(np.array(betas)*24,ax = ax,axlabel = r"estimated beta ($mm^{2}.\mu m^{-1}.day^{-1}$)",label=plate)

In [180]:
np.mean(boot_interc)

-4.787780575701489e-06

In [59]:
y, X = dmatrices('branch ~ n + rho ', data=table_rhos, return_type='dataframe')

In [60]:
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                 branch   R-squared:                       0.697
Model:                            OLS   Adj. R-squared:                  0.695
Method:                 Least Squares   F-statistic:                     327.7
Date:                Mon, 10 Jan 2022   Prob (F-statistic):           1.31e-74
Time:                        22:37:17   Log-Likelihood:                 1440.8
No. Observations:                 288   AIC:                            -2876.
Df Residuals:                     285   BIC:                            -2865.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0002      0.000     -1.426      0.1

In [30]:
y, X = dmatrices('an_rate ~  nrho ', data=table_rhos, return_type='dataframe')
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                an_rate   R-squared:                         nan
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Wed, 05 Jan 2022   Prob (F-statistic):                nan
Time:                        16:57:31   Log-Likelihood:                    inf
No. Observations:                 288   AIC:                              -inf
Df Residuals:                     286   BIC:                              -inf
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept           0          0        nan        n

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\statsmodels\regression\linear_model.py:1738: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\statsmodels\regression\linear_model.py:1827: RuntimeWarning: invalid value encountered in double_scalars
  return self.mse_model/self.mse_resid
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\statsmodels\regression\linear_model.py:926: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\statsmodels\stats\stattools.py:50: RuntimeWarning: invalid value encountered in double_scalars
  dw = np.sum(diff_resids**2, axis=axis) / np.sum(resids**2, axis=axis)


In [34]:
0.053/2.43e-5

2181.0699588477364

In [87]:
y, X = dmatrices('an_rate ~  n ', data=table_rhos, return_type='dataframe')
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                an_rate   R-squared:                       0.227
Model:                            OLS   Adj. R-squared:                  0.226
Method:                 Least Squares   F-statistic:                     463.5
Date:                Thu, 30 Dec 2021   Prob (F-statistic):           2.24e-90
Time:                        16:31:53   Log-Likelihood:                 8437.0
No. Observations:                1584   AIC:                        -1.687e+04
Df Residuals:                    1582   BIC:                        -1.686e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.427e-05   3.25e-05     -0.439      0.6

In [92]:
y, X = dmatrices('an_rate ~  n +  nrho', data=table_rhos, return_type='dataframe')
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                an_rate   R-squared:                       0.274
Model:                            OLS   Adj. R-squared:                  0.273
Method:                 Least Squares   F-statistic:                     298.5
Date:                Thu, 30 Dec 2021   Prob (F-statistic):          1.06e-110
Time:                        16:34:02   Log-Likelihood:                 8487.2
No. Observations:                1584   AIC:                        -1.697e+04
Df Residuals:                    1581   BIC:                        -1.695e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -7.295e-05    3.2e-05     -2.277      0.0

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.formula.api as sm# synthetic sample data
fig, ax = plt.subplots()

n_points = 1000
x = np.linspace(0, 10, n_points)
y = x + (np.random.rand(len(x)) * 5)
data_df = pd.DataFrame({'x': x, 'y': y})
ols_model = sm.ols(formula = 'y ~ x', data=data_df)
results = ols_model.fit()# coefficients
print('Intercept, x-Slope : {}'.format(results.params))
y_pred = ols_model.fit().predict(data_df['x'])# plot results
ax.scatter(x, y)
ax.plot(x, y_pred, linewidth=2)
ax.grid(True)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('x vs y')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Intercept, x-Slope : Intercept    2.474894
x            1.006328
dtype: float64


Text(0.5, 1.0, 'x vs y')

In [6]:
# resample with replacement each row
boot_slopes = []
boot_interc = []
n_boots = 100
fig, ax = plt.subplots()
for _ in range(n_boots):
    # sample the rows, same size, with replacement
    sample_df = data_df.sample(n=n_points, replace=True)
    # fit a linear regression
    ols_model_temp = sm.ols(formula = "y ~ x", data=sample_df)
    results_temp = ols_model_temp.fit()

    # append coefficients
    boot_interc.append(results_temp.params[0])
    boot_slopes.append(results_temp.params[1])

    # plot a greyed out line
    y_pred_temp = ols_model_temp.fit().predict(sample_df["x"])
    ax.plot(sample_df["x"], y_pred_temp, color="grey", alpha=0.2)# add data points
ax.scatter(x, y)
ax.plot(x, y_pred, linewidth=2)
ax.grid(True)
ax.set_xlabel("x")
ax.set_ylabel("x")
ax.set_title("x vs y")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'x vs y')

In [17]:
import seaborn as sns
sns.distplot(boot_slopes)
plt.show()

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
